In [1]:
%%capture
%pip install transformers trl

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="openai-community/gpt2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [3]:
outs = pipe('My name is Ellie and', num_return_sequences = 5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [4]:
type(outs)

list

In [5]:
from datasets import load_dataset

ds = load_dataset("econnors/revelatory")

revalatory_threads.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/884 [00:00<?, ? examples/s]

In [6]:
type(ds)

datasets.dataset_dict.DatasetDict

In [7]:
ds["train"]["comment"][0]

'522798765 Nice coon hand signals'

In [8]:
ds["train"]['comment'][:20]

['522798765 Nice coon hand signals',
 '522773939 white men with non white wives are a totally insufferable pair i live on a navy base but active duty coast guard and of course navy men are most susceptible to interracial marriage because they get stationed overseas i have 3 neighbors one has a little goblina wife the other two have flip wives the men are loud annoying cringe and verbose the women are too it is a wonder to behold',
 "522774752 No because its not true Unless you're going to keep White women in chains they will always find a way to undermine their own men",
 "brown people in this thread only go to Iceland to see what peak white means everyone sleeps with everyone state is super solid and takes care of everyone children need not fear anything it's paradise on earth if you're monogamous fine be it but don't force anyone into your religion laws you fucking slave i like fucking around with beautiful girls and i also want to procreate what's wrong with that i have 7 children f

In [9]:
pipe(ds["train"]['comment'][10])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '522775833 kek well said but to be fair the semi retarded rural third worlder is not seeking a hot man to fuck Rather she is looking exclusively for money You used Filipinas for your example but the same applies with any thirdie woman You will be expected to foot the bill and pay for her family indefinitely She will expect you to regularly send money back home to her parents and 20 siblings Only if you cannot provide this will she leave you But in truth even with the thirdie woman you\'re still saving money in the long term I asked co pilot how much the average divorced man pays in alimony and child support I know I know AI slop stfu niggers and it said According to the U S Census Bureau Average monthly child support payment 441 Annual total 441 12 5 292 assuming One child with payments for 18 years Lifetime child support cost 5 292 18 95 256 If there are multiple children this amount could be significantly higher Typical monthly alimony Ranges from 0 to 1 381 depen

In [10]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    pipeline
)
from trl import SFTTrainer, SFTConfig

In [11]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125m")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [12]:
dataset = load_dataset("econnors/revelatory")

In [13]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [14]:
training_params = SFTConfig(
    output_dir="./results",
    num_train_epochs = 3, # how many times we iterate over the dataset as a whole
    learning_rate = 2e-4, # how many "steps" we take in adjusting the parameters to make up for loss
    weight_decay = 0.001, # way of regularizing the parameters
    dataset_text_field = "comment",
    report_to="none" # this is a new param, to avoid a login to W&B
)

In [15]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset['train'],
    processing_class = tokenizer,
    args = training_params
)

Adding EOS to train dataset:   0%|          | 0/884 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/884 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/884 [00:00<?, ? examples/s]

In [16]:
trainer.train()


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.


Step,Training Loss
10,4.852800
20,4.818400
30,4.906100
40,4.775900
50,4.879500
60,4.805100
70,4.727200
80,4.791300
90,4.870400
100,4.758300


TrainOutput(global_step=333, training_loss=3.864245669619815, metrics={'train_runtime': 409.5096, 'train_samples_per_second': 6.476, 'train_steps_per_second': 0.813, 'total_flos': 423663627128832.0, 'train_loss': 3.864245669619815, 'entropy': 3.4871230125427246, 'num_tokens': 334482.0, 'mean_token_accuracy': 0.39926527937253314, 'epoch': 3.0})

In [17]:
pipe("I'm so excited")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I\'m so excited for you!"\n\n"I really feel like I\'m in a big fight!"\n\n"Oh my god!"\n\n"I\'m going to be in such a big fight!"\n\nI was so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\nI\'m so excited for you!\n\n'}]

In [18]:
trainer.model.save_pretrained("models")
trainer.tokenizer.save_pretrained("models")

model = AutoModelForCausalLM.from_pretrained("./models")
tokenizer = AutoTokenizer.from_pretrained("./models")

pipe = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=50)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Device set to use cuda:0


In [19]:
pipe("Gender")

[{'generated_text': 'Gendericalism vs Marxism vs Occultism vs Marxism The True Self vs Occultism The True Self vs Occultism The True Self vs Progressive Capitalism Capitalism vs Progressive Capitalism Capitalism vs Progressive Capitalism I am here to argue that the True Self is a'}]

In [20]:
pipe("Incel")

[{'generated_text': 'Incel bros are women of the cloth who wear cloth garments and have no jewish jewish affiliation with them Women who have no affiliation with them are not allowed to enter the temple or appear in the temple halls They are prohibited from entering the temple'}]